In [ ]:
#----integrating R code in python---------

import rpy2.robjects.packages as rpackages
from rpy2.robjects import numpy2ri

numpy2ri.activate()
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1) # select the first mirror in the list
utils.chooseCRANmirror(ind=1)
packnames = ('densratio',)
from rpy2.robjects.vectors import StrVector
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))

In [ ]:
!pip install git+https://github.com/hoxo-m/densratio_py.git


  Cloning https://github.com/hoxo-m/densratio_py.git to /tmp/pip-req-build-was4e2px
  Running command git clone -q https://github.com/hoxo-m/densratio_py.git /tmp/pip-req-build-was4e2px


In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils
import torchvision.transforms as transforms
from densratio import densratio
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.applications.resnet import ResNet50
from tensorflow.keras.applications import VGG16, MobileNetV2, DenseNet121,NASNetMobile,EfficientNetB0,EfficientNetB1,EfficientNetB7,InceptionV3
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import SGD
import math, sklearn.metrics.pairwise as sk
from cvxopt import matrix, solvers
import random, sys
from sklearn import svm


In [ ]:
# pipeline to switch datasets and algotithms
dataset_name="svhn"  #mnist #cifar10c #svhn
algorithm="KMM" # KMM KLIEP DR
category=0
model="RESNET" # DENSENET , EFFICIENTNET,RESNET
dsize=26000  # size of the dataset to train


# #checking for gpu
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))




In [ ]:

if category == 0:
  if dataset_name== "mnist":
    dataset_train=tfds.load('mnist', split='train', shuffle_files=True)
    dataset_test=tfds.load('mnist_corrupted', split='test', shuffle_files=True)
  elif dataset_name== "stl":
    dataset_train=tfds.load('stl10', split='train', shuffle_files=True)
    dataset_test=tfds.load('stl10', split='test', shuffle_files=True)

  elif dataset_name=="cifar10c":
    dataset_train=tfds.load('cifar10', split='train', shuffle_files=True)
    dataset_test=tfds.load('cifar10_corrupted', split='test', shuffle_files=True)
  elif dataset_name== "svhn":
    dataset_train=tfds.load('svhn_cropped', split='train', shuffle_files=True)
    dataset_test=tfds.load('svhn_cropped', split='test', shuffle_files=True)
    
  
  x_train=[]
  y_train=[]
  x_test=[]
  y_test=[]
  for ex in dataset_train:
    x_train.append(ex['image'])
    y_train.append(ex['label'])
  x_train=np.array(x_train)
  y_train=np.array(y_train)
  for ex in dataset_test:
    x_test.append(ex['image'])
    y_test.append(ex['label'])
  x_test=np.array(x_test)
  y_test=np.array(y_test)

# else:
#   dataset=get_dataset(dataset=dataset_name,download=True)
#   dataset_train = dataset.get_subset('train',transform=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()]))
#   train_loader = get_train_loader('standard', dataset_train, batch_size=16)
#   test_data = dataset.get_subset('test',transform=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()]))
  
#   test_loader = get_eval_loader('standard', test_data, batch_size=16)


 
  

  






NameError: ignored

In [ ]:
def prep_pixels(data):
  data = data.astype('float32')
  data = data/ 255.0
  return data




def reshape(data):
  if dataset_name=="mnist":
    data= data.reshape((data.shape[0], 28, 28, 1))
  elif dataset_name=="cifar10c":
    data= data.reshape((data.shape[0], 32, 32, 3))
  elif dataset_name=="svhn":
    data= data.reshape((data.shape[0], 32, 32, 3))
  elif dataset_name=="stl":
    data= data.reshape((data.shape[0], 96, 96, 3))
  return data


def compute_density_mnist(x_train,x_test):
    x_train=x_train.reshape(x_train.shape[0],784)
    x_test=x_test.reshape(x_test.shape[0],784)
    sigma=[100]
    lambda_r=[0.010]
    densratio_obj = densratio(x_train,x_test,alpha=0,sigma_range=sigma,lambda_range=lambda_r)
    train=[]
    train=np.array(densratio_obj.compute_density_ratio(x_train))
    data=[]
    for i in range(train.shape[0]):
      data.append(train[i]*x_train[i])
    return np.array(data)


def compute_density_cifar(x_train,x_test):
    x_train=x_train.reshape(x_train.shape[0],3072)
    x_test=x_test.reshape(x_test.shape[0],3072)
    sigma=[100]
    lambda_r=[1]
    densratio_obj = densratio(x_train,x_test,alpha=0,sigma_range=sigma,lambda_range=lambda_r)
    train=[]
    train=np.array(densratio_obj.compute_density_ratio(x_train))
    data=[]
    for i in range(train.shape[0]):
      data.append(train[i]*x_train[i])
    return np.array(data)


def compute_density_svhn(x_train,x_test):
    x_train=x_train.reshape(x_train.shape[0],3072)
    x_test=x_test.reshape(x_test.shape[0],3072)
    densratio_obj = densratio(x_train,x_test,alpha=0)
    train=[]
    train=np.array(densratio_obj.compute_density_ratio(x_train))
    data=[]
    for i in range(train.shape[0]):
      data.append(train[i]*x_train[i])
    return np.array(data)



def compute_density_stl(x_train,x_test):
    x_train=x_train.reshape(x_train.shape[0],27648)
    x_test=x_test.reshape(x_test.shape[0],27648)
    densratio_obj = densratio(x_train,x_test,alpha=0)
    train=[]
    train=np.array(densratio_obj.compute_density_ratio(x_train))
    data=[]
    for i in range(train.shape[0]):
      data.append(train[i]*x_train[i])
    return np.array(data)
  

def kmm_reshape(data):
  data = data/ 255.0
  if dataset_name=="mnist":
    data=data.reshape(data.shape[0],784)
  elif dataset_name=="cifar10c":
    data=data.reshape(data.shape[0],3072)
  elif dataset_name=="svhn":
    data=data.reshape(data.shape[0],3072)
  elif dataset_name=="stl":
    data=data.reshape(data.shape[0],27648)

  data=data.astype(np.double)
  return data

def kmm_reshape_back(data):
  if dataset_name=="mnist":
    data=data.reshape(data.shape[0],28,28,1)
  elif dataset_name=="cifar10c":
    data=data.reshape(data.shape[0],32,32,3)
  elif dataset_name=="svhn":
    data=data.reshape(data.shape[0],32,32,3)
  elif dataset_name=="stl":
    data=data.reshape(data.shape[0],96,96,3)
  return data



def kmm(Xtrain, Xtest, sigma):
    n_tr = len(Xtrain)
    n_te = len(Xtest)

    # calculate Kernel
    print('Computing kernel for training data ...')
    K_ns = sk.rbf_kernel(Xtrain, Xtrain, sigma)
    # make it symmetric
    K = 0.9 * (K_ns + K_ns.transpose())

    # calculate kappa
    print('Computing kernel for kappa ...')
    kappa_r = sk.rbf_kernel(Xtrain, Xtest, sigma)
    ones = np.ones(shape=(n_te, 1))
    kappa = np.dot(kappa_r, ones)
    kappa = -(float(n_tr) / float(n_te)) * kappa

    # calculate eps
    eps = (math.sqrt(n_tr) - 1) / math.sqrt(n_tr)

    # constraints
    A0 = np.ones(shape=(1, n_tr))
    A1 = -np.ones(shape=(1, n_tr))
    A = np.vstack([A0, A1, -np.eye(n_tr), np.eye(n_tr)])
    b = np.array([[n_tr * (eps + 1), n_tr * (eps - 1)]])
    b = np.vstack([b.T, -np.zeros(shape=(n_tr, 1)), np.ones(shape=(n_tr, 1)) * 1000])

    print('Solving quadratic program for beta ...')
    P = matrix(K, tc='d')
    q = matrix(kappa, tc='d')
    G = matrix(A, tc='d')
    h = matrix(b, tc='d')
    beta = solvers.qp(P, q, G, h)
    return [i for i in beta['x']]


def computeKernelWidth(data):
    dist = []
    for i in range(len(data)):
        for j in range(i + 1, len(data)):
            # s = self.__computeDistanceSq(data[i], data[j])
            # dist.append(math.sqrt(s))
            dist.append(np.sqrt(np.sum((np.array(data[i]) - np.array(data[j])) ** 2)))
    return np.median(np.array(dist))


def getFixedBeta(value, count):
    beta = []
    for c in range(count):
        beta.append(value)
    return beta


def getBeta(trainX, testX, maxvar):
    beta = []
    # gammab = 0.001
    gammab = computeKernelWidth(trainX)
    print("Gammab:", gammab)

    beta = kmm(trainX, testX, gammab)
    print("{0} Beta: {1}".format(len(beta), beta))

    return beta



#--------------------KMM-------------------
def kmm_training_set_mnist(data1,data2):
  beta_weight=getBeta(data1,data2,784)
  data=[]
  for i in range(len(beta_weight)):
    data.append(beta_weight[i]*data1[i])
  data=np.array(data)
  return data


def kmm_training_set_cifar(data1,data2):
  beta_weight=getBeta(data1,data2,3072)
  data=[]
  for i in range(len(beta_weight)):
    data.append(beta_weight[i]*data1[i])
  data=np.array(data)
  return data

def kmm_training_set_svhn(data1,data2):
  beta_weight=getBeta(data1,data2,3072)
  data=[]
  for i in range(len(beta_weight)):
    data.append(beta_weight[i]*data1[i])
  data=np.array(data)
  return data


def kmm_training_set_stl(data1,data2):
  beta_weight=getBeta(data1,data2,27648)
  data=[]
  for i in range(len(beta_weight)):
    data.append(beta_weight[i]*data1[i])
  data=np.array(data)
  return data




#-----------------kliep------------------
def kliep(data1,data2):
  if dataset_name=="mnist":
    data1=data1.reshape(data1.shape[0],784)
    data2=data2.reshape(data2.shape[0],784)
  elif dataset_name=="cifar10c":
    data1=data1.reshape(data1.shape[0],3072)
    data2=data2.reshape(data2.shape[0],3072)
  elif dataset_name=="svhn":
    data1=data1.reshape(data1.shape[0],3072)
    data2=data2.reshape(data2.shape[0],3072)
  elif dataset_name=="stl":
    data1=data1.reshape(data1.shape[0],27648)
    data2=data2.reshape(data2.shape[0],27648)
  densratio_r=rpackages.importr('densratio').densratio
  densratio_obj_r=densratio_r(data1,data2,method="KLIEP")
  return_set=np.array(densratio_obj_r[3](data1))
  data=[]
  for i in range(return_set.shape[0]):
    data.append(return_set[i]*data1[i])
  return np.array(data)

def prep_y_labels(data):
  return tf.keras.utils.to_categorical(data , num_classes=10)


def nn_reshape(data):
  data_new=[]
  for ex in data:
    ex=np.pad(ex, ((2,2),(2,2),(0,0)), 'constant')
    dim = np.zeros((32,32))
    ex = np.stack((ex.reshape(32,32),dim, dim), axis=2)
    data_new.append(ex)
  
  return np.array(data_new)


In [ ]:
#------weighted density ratio--------
if dataset_name=="mnist":
  if algorithm=="DR":

      x_train=prep_pixels(x_train)
      x_test=prep_pixels(x_test)
      x_train_new=compute_density_mnist(x_train[:dsize],x_test[:dsize])
      x_train=reshape(x_train)
      x_test=reshape(x_test)
      x_train_new=reshape(x_train_new)

  elif algorithm=="KMM":
      x_train=kmm_reshape(x_train)
      x_test=kmm_reshape(x_test)
      x_train_new=kmm_training_set_mnist(x_train[:dsize],x_test[:dsize])
      x_train=kmm_reshape_back(x_train)
      x_test=kmm_reshape_back(x_test)
      x_train_new=kmm_reshape_back(x_train_new)
  elif algorithm=="KLIEP":
      x_train=prep_pixels(x_train)
      x_test=prep_pixels(x_test)
      x_train_new=kliep(x_train[:dsize],x_test[:dsize])
      x_train=reshape(x_train)
      x_test=reshape(x_test)
      x_train_new=reshape(x_train_new)
  y_train=prep_y_labels(y_train)
  y_test=prep_y_labels(y_test)


elif dataset_name=="cifar10c":
  if algorithm=="DR":

      x_train=prep_pixels(x_train)
      x_test=prep_pixels(x_test)
      x_train_new=compute_density_cifar(x_train[:dsize],x_test[:dsize])
      x_train=reshape(x_train)
      x_test=reshape(x_test)
      x_train_new=reshape(x_train_new)
  elif algorithm=="KMM":
      x_train=kmm_reshape(x_train)
      x_test=kmm_reshape(x_test)
      x_train_new=kmm_training_set_cifar(x_train[:dsize],x_test[:dsize])
      x_train=kmm_reshape_back(x_train)
      x_test=kmm_reshape_back(x_test)
      x_train_new=kmm_reshape_back(x_train_new)
  elif algorithm=="KLIEP":
      x_train=prep_pixels(x_train)
      x_test=prep_pixels(x_test)
      x_train_new=kliep(x_train[:dsize],x_test[:dsize])
      x_train=reshape(x_train)
      x_test=reshape(x_test)
      x_train_new=reshape(x_train_new)
  y_train=prep_y_labels(y_train)
  y_test=prep_y_labels(y_test)


elif dataset_name=="svhn":
  if algorithm=="DR":

      x_train=prep_pixels(x_train)
      x_test=prep_pixels(x_test)
      x_train_new=compute_density_svhn(x_train[:dsize],x_test[:dsize])
      x_train=reshape(x_train)
      x_test=reshape(x_test)
      x_train_new=reshape(x_train_new)
  elif algorithm=="KMM":
      x_train=kmm_reshape(x_train)
      x_test=kmm_reshape(x_test)
      x_train_new=kmm_training_set_svhn(x_train[:dsize],x_test[:dsize])
      x_train=kmm_reshape_back(x_train)
      x_test=kmm_reshape_back(x_test)
      x_train_new=kmm_reshape_back(x_train_new)
  elif algorithm=="KLIEP":
      x_train=prep_pixels(x_train)
      x_test=prep_pixels(x_test)
      x_train_new=kliep(x_train[:dsize],x_test[:dsize])
      x_train=reshape(x_train)
      x_test=reshape(x_test)
      x_train_new=reshape(x_train_new)
  y_train=prep_y_labels(y_train)
  y_test=prep_y_labels(y_test)


elif dataset_name=="stl":
  if algorithm=="DR":

      x_train=prep_pixels(x_train)
      x_test=prep_pixels(x_test)
      x_train_new=compute_density_stl(x_train[:dsize],x_test[:dsize])
      x_train=reshape(x_train)
      x_test=reshape(x_test)
      x_train_new=reshape(x_train_new)
  elif algorithm=="KMM":
      x_train=kmm_reshape(x_train)
      x_test=kmm_reshape(x_test)
      x_train_new=kmm_training_set_stl(x_train[:dsize],x_test[:dsize])
      x_train=kmm_reshape_back(x_train)
      x_test=kmm_reshape_back(x_test)
      x_train_new=kmm_reshape_back(x_train_new)
  elif algorithm=="KLIEP":
      x_train=prep_pixels(x_train)
      x_test=prep_pixels(x_test)
      x_train_new=kliep(x_train[:dsize]/10,x_test[:dsize]/10)
      x_train=reshape(x_train)
      x_test=reshape(x_test)
      x_train_new=reshape(x_train_new)
  y_train=prep_y_labels(y_train)
  y_test=prep_y_labels(y_test)

  

  
    



 

Gammab: 13.22263330529649
Computing kernel for training data ...
Computing kernel for kappa ...


In [ ]:
plt.imshow(x_train_new[0]*100)

NameError: ignored

In [ ]:
# x_train_new1=np.load('/content/SVHN_New_DRO_train_X.npy')
# x_test1=np.load('/content/SVHN_New_DRO_test_X.npy')
# y_train1=np.load('/content/SVHN_New_DRO_train_Y.npy')
# y_test1=np.load('/content/SVHN_New_DRO_test_Y.npy')




In [ ]:
if dataset_name=="mnist":
  x_train=nn_reshape(x_train)
  x_test=nn_reshape(x_test)
  x_train_new=nn_reshape(x_train_new)
if algorithm=="KMM":
  x_train_new=x_train_new*100


In [ ]:
#-----model training-------

batch=32
epoch_val=10
if dataset_name=="stl":
	insh=96
else:
	insh=32
def define_model():
	model = Sequential()
	model.add(Conv2D(96, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(insh, insh, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

nn_model=define_model()


if algorithm=="UW":
  nn_model.fit(x_train[:26000], y_train[:26000], batch_size=batch, epochs=epoch_val, verbose = 2)
else:
  nn_model.fit(x_train_new, y_train[:len(x_train_new)], batch_size=batch, epochs=epoch_val, verbose = 2)


nn_model.evaluate(x_test,y_test)



Epoch 1/10
157/157 - 2s - loss: 4.1043 - accuracy: 0.3248
Epoch 2/10
157/157 - 1s - loss: 1.1994 - accuracy: 0.5698
Epoch 3/10
157/157 - 1s - loss: 0.7102 - accuracy: 0.7518
Epoch 4/10
157/157 - 1s - loss: 0.3753 - accuracy: 0.8784
Epoch 5/10
157/157 - 1s - loss: 0.1801 - accuracy: 0.9472
Epoch 6/10
157/157 - 1s - loss: 0.0680 - accuracy: 0.9816
Epoch 7/10
157/157 - 1s - loss: 0.0659 - accuracy: 0.9820
Epoch 8/10
157/157 - 1s - loss: 0.0301 - accuracy: 0.9938
Epoch 9/10
157/157 - 1s - loss: 0.0174 - accuracy: 0.9962
Epoch 10/10
157/157 - 1s - loss: 0.0041 - accuracy: 0.9996
250/250 [==============================] - 1s 5ms/step - loss: 2.5717 - accuracy: 0.4841


[2.571734666824341, 0.4841249883174896]

In [ ]:
model="DENSENET" # DENSENET , EFFICIENTNET,RESNET


In [ ]:

#--------------------
if dataset_name=="stl":
	insh=96
else:
	insh=32
input = tf.keras.Input(shape=(insh,insh,3))

#resnet
if model=="RESNET":
  fmod = tf.keras.applications.ResNet50(weights='imagenet',
                                             include_top = False, 
                                             input_tensor = input)
  
#densenet 
elif model == "DENSENET":
  fmod=tf.keras.applications.DenseNet121(
      weights='imagenet',include_top=False,input_tensor=input
  )



gap = tf.keras.layers.GlobalMaxPooling2D()(fmod.output)

output = tf.keras.layers.Dense(10, activation='softmax', use_bias=True)(gap)

func_model = tf.keras.Model(fmod.input, output)





In [ ]:
#--------------------------------
batch=32
epoch_val=10

func_model.compile(
          loss  = tf.keras.losses.CategoricalCrossentropy(),
          metrics = tf.keras.metrics.CategoricalAccuracy(),
          optimizer = tf.keras.optimizers.Adam())



if algorithm=="UW":
  func_model.fit(x_train[:26000], y_train[:26000], batch_size=batch, epochs=epoch_val, verbose = 2)
else:
  func_model.fit(x_train_new, y_train[:len(x_train_new)], batch_size=batch, epochs=epoch_val, verbose = 2)

func_model.evaluate(x_test,y_test)

Epoch 1/10
157/157 - 24s - loss: 1.4088 - categorical_accuracy: 0.6202
Epoch 2/10
157/157 - 12s - loss: 0.6187 - categorical_accuracy: 0.7912
Epoch 3/10
157/157 - 12s - loss: 0.4697 - categorical_accuracy: 0.8470
Epoch 4/10
157/157 - 12s - loss: 0.6681 - categorical_accuracy: 0.7848
Epoch 5/10
157/157 - 12s - loss: 0.3456 - categorical_accuracy: 0.8840
Epoch 6/10
157/157 - 12s - loss: 0.2019 - categorical_accuracy: 0.9272
Epoch 7/10
157/157 - 12s - loss: 0.1753 - categorical_accuracy: 0.9462
Epoch 8/10
157/157 - 12s - loss: 0.2406 - categorical_accuracy: 0.9250
Epoch 9/10
157/157 - 12s - loss: 0.1626 - categorical_accuracy: 0.9432
Epoch 10/10
157/157 - 12s - loss: 0.2420 - categorical_accuracy: 0.9200
250/250 [==============================] - 10s 28ms/step - loss: 1.5917 - categorical_accuracy: 0.6674


[1.5917105674743652, 0.6673750281333923]